In [1]:
!pip install librosa --user
!pip install sndfile
import os
import subprocess
import sagemaker
import torch
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

_role = get_execution_role()


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


## 准备数据

In [ ]:
import librosa
import json
sample_rate=16000
def generate_json(in_dir,out_dir,out_filename,ttype):
    file_infos = []
    in_dir = os.path.abspath(in_dir)
    wav_list = os.listdir(in_dir)
    for wav_file in wav_list:
        if not wav_file.endswith('.wav'):
            continue
        wav_path = os.path.join(in_dir, wav_file)
        samples, _ = librosa.load(wav_path, sr=sample_rate)
        fake_wav_path = os.path.join('/opt/ml/input/data', ttype, out_filename, wav_file)
        file_infos.append((fake_wav_path, len(samples)))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    with open(os.path.join(out_dir, out_filename + '.json'), 'w') as f:
        json.dump(file_infos, f, indent=4)
        
generate_json('train_sample_v6/tr/mix_both','train_sample_v6/tr','mix_both','train')
generate_json('train_sample_v6/tr/s1','train_sample_v6/tr','s1','train')
generate_json('train_sample_v6/tr/s2','train_sample_v6/tr','s2','train')
generate_json('train_sample_v6/cv/mix_both','train_sample_v6/cv','mix_both','test')
generate_json('train_sample_v6/cv/s1','train_sample_v6/cv','s1','test')
generate_json('train_sample_v6/cv/s2','train_sample_v6/cv','s2','test')

!aws s3 sync train_sample_v6/tr/ s3://sagemaker-us-east-1-002224604296/separate/train
!aws s3 sync train_sample_v6/cv/ s3://sagemaker-us-east-1-002224604296/separate/test

## 开始训练和部署流程
### 设置训练和验证数据路径

In [2]:
# # #anbei
# train_inputs="s3://sagemaker-us-east-2-310850127430/shouqi/train/"
# test_inputs="s3://sagemaker-us-east-2-310850127430/shouqi/test/"
#aoyu
train_inputs="s3://sagemaker-us-east-1-002224604296/separate/train/"
test_inputs="s3://sagemaker-us-east-1-002224604296/separate/test/"

### 利用sagemaker pytorch进行训练

In [3]:
from sagemaker.pytorch import PyTorch

dprnn_estimator = PyTorch(entry_point='dprnn-train-deploy.py',
                            source_dir='code',
                            train_instance_type='ml.p3.2xlarge',
                            train_instance_count=1,
                            role=_role,
                            framework_version='1.5.1',
                            hyperparameters = {'epochs': 1})
dprnn_estimator.fit({'train': train_inputs,
                       'test': test_inputs})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-06-24 07:11:36 Starting - Starting the training job...
2020-06-24 07:11:38 Starting - Launching requested ML instances.........
2020-06-24 07:13:26 Starting - Preparing the instances for training......
2020-06-24 07:14:27 Downloading - Downloading input data...
2020-06-24 07:14:48 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-24 07:16:16,243 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-06-24 07:16:16,265 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-24 07:16:16,272 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-24 07:16:16,575 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.txt

2020-06-24 07:16:15 Training - Training image download com

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.12
    Uninstalling Cython-0.29.12:
      Successfully uninstalled Cython-0.29.12
2020-06-24 07:16:37,879 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_pytorch_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "epochs": 1
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "test": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        },
        "train": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "Non


2020-06-24 07:17:24 Completed - Training job completed
Training seconds: 177
Billable seconds: 177


### 将训练好的模型进行部署

In [4]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = dprnn_estimator.deploy(instance_type='ml.g4dn.xlarge',
                                     initial_instance_count=1)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

### 在部署好的模型上进行推理

In [6]:
from helper.utils import *

# 10 seconds for each 
instance_time = 10
filename = './audio_raw_1-20_clip_0.wav'
# filename = './data/tt/mix/1-20-t-1_1-20-t-1_0dB.wav'

x, fs = preprocess_wav(filename)
instance_seg = instance_time * fs

In [ ]:
l_x = len(x)
s_x = instance_seg
s1 = np.array([])
s2 = np.array([])
gl_i = 0
for start in range(0, l_x-s_x, s_x):
    flat_estimate = separate_process(x[start:start+s_x], predictor)
    s1 = np.concatenate([s1, flat_estimate[0]])
    s2 = np.concatenate([s2, flat_estimate[1]])
    gl_i = start+s_x
if gl_i < l_x:
    flat_estimate = separate_process(x[gl_i:], predictor)
    s1 = np.concatenate([s1, flat_estimate[0]])
    s2 = np.concatenate([s2, flat_estimate[1]])
    
write(s1, './complete_split_s1.wav')
write(s2, './complete_split_s2.wav')

In [14]:
import argparse

import librosa
import torch

from scipy.signal import lfilter
import numpy as np
import scipy.io.wavfile
import soundfile as sf

import fnmatch, os, warnings

def remove_pad(inputs, inputs_lengths):
    """
    Args:
        inputs: torch.Tensor, [B, C, T] or [B, T], B is batch size
        inputs_lengths: torch.Tensor, [B]
    Returns:
        results: a list containing B items, each item is [C, T], T varies
    """
    results = []
    dim = inputs.dim()
    if dim == 3:
        C = inputs.size(1)
    for input, length in zip(inputs, inputs_lengths):
        if dim == 3: # [B, C, T]
            results.append(input[:,:length].view(C, -1).cpu().numpy())
        elif dim == 2:  # [B, T]
            results.append(input[:length].view(-1).cpu().numpy())
    return results


def write(inputs, filename, sr=16000):
    librosa.output.write_wav(filename, inputs, sr)

def wavread(filename):
    fs, x = scipy.io.wavfile.read(filename)
    if np.issubdtype(x.dtype, np.integer):
        x = x / np.iinfo(x.dtype).max
    return x, fs

def wavwrite(filename, s, fs):
    if s.dtype != np.int16:
        s = np.array(s * 2**15, dtype=np.int16)
    if np.any(s > np.iinfo(np.int16).max) or np.any(s < np.iinfo(np.int16).min):
        warnings.warn('Warning: clipping detected when writing {}'.format(filename))
    scipy.io.wavfile.write(filename, fs, s)

def asl_meter(x, fs, nbits=16):
    '''Measure the Active Speech Level (ASR) of x following ITU-T P.56.
    If x is integer, it will be scaled to (-1, 1) according to nbits.
    '''

    if np.issubdtype(x.dtype, np.integer):
        x = x / 2**(nbits-1)

    # Constants
    MIN_LOG_OFFSET = 1e-20
    T = 0.03                # Time constant of smoothing in seconds
    g = np.exp(-1/(T*fs))
    H = 0.20                # Time of handover in seconds
    I = int(np.ceil(H*fs))
    M = 15.9                # Margin between threshold and ASL in dB

    a = np.zeros(nbits-1)                       # Activity count
    c = 0.5**np.arange(nbits-1, 0, step=-1)     # Threshold level
    h = np.ones(nbits)*I                        # Hangover count
    s = 0
    sq = 0
    p = 0
    q = 0
    asl = -100

    L = len(x)
    s = sum(abs(x))
    sq = sum(x**2)
    dclevel = s/np.arange(1, L+1)
    lond_term_level = 10*np.log10(sq/np.arange(1, L+1) + MIN_LOG_OFFSET)
    c_dB = 20*np.log10(c)

    for i in range(L):
        p = g * p + (1-g) * abs(x[i])
        q = g * q + (1-g) * p

        for j in range(nbits-1):
            if q >= c[j]:
                a[j] += 1
                h[j] = 0
            elif h[j] < I:
                a[j] += 1
                h[j] += 1

    a_dB = -100 * np.ones(nbits-1)

    for i in range(nbits-1):
        if a[i] != 0:
            a_dB[i] = 10*np.log10(sq/a[i])

    delta = a_dB - c_dB
    idx = np.where(delta <= M)[0]

    if len(idx) != 0:
        idx = idx[0]
        if idx > 1:
            asl = bin_interp(a_dB[idx], a_dB[idx-1], c_dB[idx], c_dB[idx-1], M)
        else:
            asl = a_dB[idx]

    return asl

def bin_interp(upcount, lwcount, upthr, lwthr, margin, tol=0.1):
    n_iter = 1
    if abs(upcount - upthr - margin) < tol:
        midcount = upcount
    elif abs(lwcount - lwthr - margin) < tol:
        midcount = lwcount
    else:
        midcount = (upcount + lwcount)/2
        midthr = (upthr + lwthr)/2
        diff = midcount - midthr - margin
        while abs(diff) > tol:
            n_iter += 1
            if n_iter > 20:
                tol *= 1.1
            if diff > tol:
                midcount = (upcount + midcount)/2
                midthr = (upthr + midthr)/2
            elif diff < -tol:
                midcount = (lwcount + midcount)/2
                midthr = (lwthr + midthr)/2
            diff = midcount - midthr - margin
    return midcount


def rms_energy(x):
    return 10*np.log10((1e-12 + x.dot(x))/len(x))

def preprocess_wav(filename):
    # filename = './1-20.wav'
    x, fs = wavread(filename)
    x_mirror = x
    # x = np.float32(x)
    # print(x.dtype)
    # N_dB = rms_energy(x_mirror)
    # S_dB = asl_meter(x, fs)
    # print(N_dB)
    # print(S_dB)

    # N_new = S_dB
    # x_mirror = 10**(N_new/20) * x_mirror / 10**(N_dB/20)
    asl_level = -26.0

    y = x + x_mirror
    y = y/10**(asl_meter(y, fs)/20) * 10**(asl_level/20)
    return x, fs

def pad_list(xs, pad_value):
    n_batch = len(xs)
    max_len = max(x.size(0) for x in xs)
    pad = xs[0].new(n_batch, max_len, * xs[0].size()[1:]).fill_(pad_value)
    for i in range(n_batch):
        pad[i, :xs[i].size(0)] = xs[i]
    return pad

def separate_process(x, predictor):
    ilens = np.array([x.shape[0]])

    # perform padding and convert to tensor
    pad_value = 0
    mixtures_pad = pad_list([torch.from_numpy(x).float()], pad_value)
    ilens = torch.from_numpy(ilens) 
    mixture_np = mixtures_pad.numpy()
    mix_lengths = ilens.cuda()
    # Forward
    response = predictor.predict(mixture_np)
#     print(response)
#     estimate_source = response.argmax(axis=1)[0]
    response = torch.from_numpy(response) 
    estimate_source = response.cuda()
    # Remove padding and flat
#     print(estimate_source)
    flat_estimate = remove_pad(estimate_source, mix_lengths)

#     with torch.no_grad():
#         mixture, mix_lengths = mixtures_pad.cuda(), ilens.cuda()

#         # Forward
#         estimate_source = model(mixture)  # [B, C, T]
        
#         # Remove padding and flat
#         flat_estimate = remove_pad(estimate_source, mix_lengths)
    return flat_estimate[0]